<div>
  <center>
    <h1>Universidad de Guadalajara</h1>
    <h2>Centro Universitario de Ciencias Económicas y Administrativas (CUCEA)</h2>
    <h2>Análisis Exploratorio de Datos EDA para el proyecto: Aplicación Web para Consulta de la Evaluación Docente.</h2>
    <b>Materia: </b> Desarrollo de Proyectos l <br>
    <b>Catedrática: </b> Mtro. Victor Hugo Cuspinera Contreras.<br>
    <b>Alumno: </b> Cristian Ulises Barenca Sotelo.<br>
    <b>Matrícula: </b> 323018977<br>
    <b>Correo:</b> cristian.barenca1897@alumnos.udg.mx <br>
  </center>
  
  <div style="text-align: right;">
    Miércoles 16 de octubre del 2024, Manzanillo, Col.
  </div>
</div>    

------

# Introducción 🚩

El EDA es una herramienta de análisis para una revisión iterativa de las bases de datos con la cual describimos las principales características usando estadística descriptiva y visualización de datos.

La presente investigación propone el desarrollo de una Aplicación Web y una Base de Datos Relacional con el objetivo de optimizar los tiempos de procesamiento en consultas de información, alcanzando un tiempo promedio de respuesta inferior a 50 segundos. Esta mejora se apoya en evidencias de la literatura que destacan el impacto positivo de las técnicas de optimización en el rendimiento del hardware de Bases de Datos. 

En este Notebook detallaremos los resultados de la exploración del análisis EDA, sigamos 🚀...

[Link de la publicación en LinkedIn](https://www.linkedin.com/posts/cristian-ulises-barenca-sotelo-50ba03183_github-cbarencaaplicacion-web-para-consulta-de-la-evaluacion-docente-activity-7252664689313091584-RBPk?utm_source=share&utm_medium=member_desktop)

---

# Contenido 📙

<a href="#1">1. Fuentes de datos</a><br>
<a href="#2">2. Descripción de los datos</a><br>
<a href="#3">3. Limpieza de datos</a><br>
<a href="#4">4. Missing values</a><br>
<a href="#5">5. Visualización de datos</a><br>
<a href="#6">6. Comentarios adicionales</a><br>
<a href="#7">7. Referencias</a><br>

---

# <a id="1">1. Fuentes de datos<a> 🔑

**Debido a la política de protección de datos, es importante señalar que los datos utilizados son sintéticos, generados a partir de ChatGPT 4o mini.**  

La fuente de información mediante la cual creamos nuestra base de datos está compuesta por datos no estructurados en formato de texto, posteriormente se desarrolló un script SQL para la inserción de la información en las diferentes tablas que componen la base de datos, este archivo queda a su disposición en la siguiente liga: <a href="https://github.com/CBarenca/Aplicacion-Web-para-Consulta-de-la-Evaluacion-Docente/blob/main/data/evdoc_insert_data.sql">clic aquí</a>.

La base de datos denominada evdoc mediante la cual realizaremos este estudio, fue creada en el año 2024, cuenta con un total de 20 tablas, mencionando algunas, ejemplo: carreras, ciclos, departamentos, generos, nrc, profesores, estudiantes,resultado_encuesta_ultimo_ciclo, roles, resultado_encuesta_historico, entre otros.

In [1]:
# Importamos las librarias necesarias para el desarrollo del análisis exploratorio de datos EDA
import mysql.connector
import pandas as pd
import numpy as np
import altair as alt
import warnings
import re
from datetime import datetime

#< Hay distintos tipos de renderers:
alt.renderers.enable('default') # <-es el más común pero no refleja las gráficas en GitHub
alt.renderers.enable('jupyterlab') # <-este nos sirve para ver las gráficas en GitHub

# Ignorar advertencias específicas
warnings.filterwarnings("ignore", category=UserWarning)  # Para ignorar UserWarnings
warnings.filterwarnings("ignore", category=FutureWarning)  # Para ignorar FutureWarnings

In [2]:
# Nos conectamos a la base de datos
conexion = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='evdoc'
)

# Creamos un cursor para realizar las consultas hacia la base de datos
cursor = conexion.cursor()

---

# <a id="2">2. Descripción de los datos<a> 📑

**Revisamos las tablas que componen la base de datos**

In [3]:
# Ejecutamos la consulta para obtener las tablas de la base de datos
cursor.execute("SHOW TABLES;")
tablas = cursor.fetchall()

# Mostramos las tablas
for tabla in tablas:
    print(tabla[0])

carreras
centros_universitarios
ciclos
departamentos
encuestas
estatus
estudiantes
generos
gestores
nacionalidades
niveles_educativos
nrc
preguntas
profesores
respuesta_preguntas
resultado_encuesta_historico
resultado_encuesta_ultimo_ciclo
roles
situaciones_academicas
tipos_contratos


**Revisamos los atributos e información que componen la tabla carreras**

In [4]:
# Revisamos como esta compuesta la información de la tabla carreras
cursor.execute('DESCRIBE carreras;')
# Obtener resultados
carreras = cursor.fetchall()
print("------ Atributos de la tabla -------")
for atributo in carreras:
    print(f"Nombre: {atributo[0]}, Tipo: {atributo[1]}, Nulo: {atributo[2]}, Clave: {atributo[3]}, Predeterminado: {atributo[4]}, Extra: {atributo[5]}")
print("------------------------------------")


#Imprimimos los valores de la tabla carreras
print("Valores de la tabla carreras")
cursor.execute('SELECT * FROM carreras;')
# Obtener resultados
valores_carreras = cursor.fetchall()
for fila in valores_carreras:
    print(fila)


------ Atributos de la tabla -------
Nombre: id_carrera, Tipo: tinyint(4), Nulo: NO, Clave: PRI, Predeterminado: None, Extra: 
Nombre: carrera, Tipo: varchar(100), Nulo: NO, Clave: , Predeterminado: None, Extra: 
------------------------------------
Valores de la tabla carreras
(1, 'Ingeniería de software')
(2, 'Ingeniería Civil')
(3, 'Arquitectura')
(4, 'Medicina')
(5, 'Derecho')
(6, 'Psicología')
(7, 'Administración de Empresas')
(8, 'Contaduría Pública')
(9, 'Ingeniería Electrónica')
(10, 'Ingeniería Industrial')
(11, 'Biología')
(12, 'Química')
(13, 'Física')
(14, 'Matemáticas')
(15, 'Ciencias de la Computación')
(16, 'Ingeniería Mecánica')
(17, 'Diseño Gráfico')
(18, 'Comunicación Social')
(19, 'Trabajo Social')
(20, 'Enfermería')


Como podemos apreciar, contamos con 20 carreras: Ingeniería de software, Ingeniería civil, arquitectura, medicina, derecho, por mencionar algunas.

**Revisamos los atributos e información que componen la tabla centros_universitarios**

In [5]:
# Revisamos como esta compuesta la información de la tabla centros_universitarios
cursor.execute('DESCRIBE centros_universitarios;')
# Obtener resultados
centros_universitarios = cursor.fetchall()
print("------ Atributos de la tabla -------")
for atributo in centros_universitarios:
    print(f"Nombre: {atributo[0]}, Tipo: {atributo[1]}, Nulo: {atributo[2]}, Clave: {atributo[3]}, Predeterminado: {atributo[4]}, Extra: {atributo[5]}")
print("------------------------------------")


#Imprimimos los valores de la tabla centros_universitarios
print("Valores de la tabla centros universitarios")
cursor.execute('SELECT * FROM centros_universitarios;')
# Obtener resultados
valores_centros_universitarios = cursor.fetchall()
for fila in valores_centros_universitarios:
    print(fila)

------ Atributos de la tabla -------
Nombre: id_centro_universitario, Tipo: tinyint(4), Nulo: NO, Clave: PRI, Predeterminado: None, Extra: 
Nombre: centro_universitario, Tipo: varchar(100), Nulo: NO, Clave: , Predeterminado: None, Extra: 
------------------------------------
Valores de la tabla centros universitarios
(1, 'Centro Universitario de Arte, Arquitectura y Diseño.')
(2, 'Centro Universitario de Ciencias Biológicas y Agropecuarias')
(3, 'Centro Universitario de Ciencias Económico Administrativas.')
(4, 'Centro Universitario de Ciencias Exactas e Ingenierías.')
(5, 'Centro Universitario de Ciencias de la Salud.')
(6, 'Centro Universitario de los Altos.')
(7, 'Centro Universitario de la Costa.')
(8, 'Centro Universitario de La Ciénega.')
(9, 'Centro Universitario de Los Lagos.')
(10, 'Centro Universitario de la Costa Sur.')
(11, 'Campus Universitario de Los Valles.')


Esta universidad cuenta con 11 centros universitarios repartidos por todo el estado de Jalisco, México. 

**Revisamos los atributos e información que componen la tabla ciclos**

In [6]:
# Revisamos como esta compuesta la información de la tabla ciclos
cursor.execute('DESCRIBE ciclos;')
# Obtener resultados
ciclos = cursor.fetchall()
print("------ Atributos de la tabla -------")
for atributo in ciclos:
    print(f"Nombre: {atributo[0]}, Tipo: {atributo[1]}, Nulo: {atributo[2]}, Clave: {atributo[3]}, Predeterminado: {atributo[4]}, Extra: {atributo[5]}")
print("------------------------------------")


#Imprimimos los valores de la tabla ciclos
print("Valores de la tabla ciclos")
cursor.execute('SELECT * FROM ciclos;')
# Obtener resultados
valores_ciclos = cursor.fetchall()
for fila in valores_ciclos:
    print(fila)


------ Atributos de la tabla -------
Nombre: id_ciclo, Tipo: tinyint(4), Nulo: NO, Clave: PRI, Predeterminado: None, Extra: 
Nombre: ciclo, Tipo: int(6), Nulo: NO, Clave: , Predeterminado: None, Extra: 
------------------------------------
Valores de la tabla ciclos
(1, 202410)


Este ciclo hace alusión al ciclo 2024A, para la eficiencia de las consultas en bases de datos relacionales es preferente utilizar números que letras.

**Revisamos los atributos e información que componen la tabla departamentos** 


In [7]:
# Revisamos como esta compuesta la información de la tabla departamentos
cursor.execute('DESCRIBE departamentos;')
# Obtener resultados
departamentos = cursor.fetchall()
print("------ Atributos de la tabla -------")
for atributo in departamentos:
    print(f"Nombre: {atributo[0]}, Tipo: {atributo[1]}, Nulo: {atributo[2]}, Clave: {atributo[3]}, Predeterminado: {atributo[4]}, Extra: {atributo[5]}")
print("------------------------------------")


#Imprimimos los valores de la tabla departamentos
print("Valores de la tabla departamentos")
cursor.execute('SELECT * FROM departamentos;')
# Obtener resultados
valores_departamentos = cursor.fetchall()
for fila in valores_departamentos:
    print(fila)

------ Atributos de la tabla -------
Nombre: id_departamento, Tipo: tinyint(4), Nulo: NO, Clave: PRI, Predeterminado: None, Extra: 
Nombre: departamentos, Tipo: varchar(100), Nulo: NO, Clave: , Predeterminado: None, Extra: 
------------------------------------
Valores de la tabla departamentos
(1, 'Recursos Humanos')
(2, 'Finanzas')
(3, 'Marketing')
(4, 'Ventas')
(5, 'Operaciones')
(6, 'Tecnologías de la Información')
(7, 'Atención al Cliente')
(8, 'Desarrollo de Productos')
(9, 'Investigación y Desarrollo')
(10, 'Logística')
(11, 'Compras')
(12, 'Calidad')
(13, 'Legal')
(14, 'Comunicación Interna')
(15, 'Formación y Capacitación')
(16, 'Estrategia Corporativa')
(17, 'Sistemas')
(18, 'Seguridad y Salud Ocupacional')
(19, 'Relaciones Públicas')
(20, 'Auditoría Interna')


En esta universidad se cuentan con 20 departamentos, los cuales albergan a los profesores y materias académicas.

**Revisamos los atributos e información que componen la tabla encuestas**

In [8]:
# Revisamos como esta compuesta la información de la tabla encuestas
cursor.execute('DESCRIBE encuestas;')
# Obtener resultados
encuestas = cursor.fetchall()
print("------ Atributos de la tabla -------")
for atributo in encuestas:
    print(f"Nombre: {atributo[0]}, Tipo: {atributo[1]}, Nulo: {atributo[2]}, Clave: {atributo[3]}, Predeterminado: {atributo[4]}, Extra: {atributo[5]}")
print("------------------------------------")


#Imprimimos los valores de la tabla carreras
print("Valores de la tabla encuestas")
cursor.execute('SELECT * FROM encuestas;')
# Obtener resultados
valores_encuestas = cursor.fetchall()
for fila in valores_encuestas:
    print(fila)

------ Atributos de la tabla -------
Nombre: id_encuesta, Tipo: tinyint(4), Nulo: NO, Clave: PRI, Predeterminado: None, Extra: 
Nombre: encuesta, Tipo: varchar(100), Nulo: NO, Clave: , Predeterminado: None, Extra: 
------------------------------------
Valores de la tabla encuestas
(1, 'E202410')


Hasta este momento solo se encuentra una encuesta registrada que corresponde al ciclo 2024A.

**Revisamos los atributos e información que componen la tabla estatus**

In [9]:
# Revisamos como esta compuesta la información de la tabla estatus
cursor.execute('DESCRIBE estatus;')
# Obtener resultados
estatus = cursor.fetchall()
print("------ Atributos de la tabla -------")
for atributo in estatus:
    print(f"Nombre: {atributo[0]}, Tipo: {atributo[1]}, Nulo: {atributo[2]}, Clave: {atributo[3]}, Predeterminado: {atributo[4]}, Extra: {atributo[5]}")
print("------------------------------------")


#Imprimimos los valores de la tabla estatus
print("Valores de la tabla estatus")
cursor.execute('SELECT * FROM estatus;')
# Obtener resultados
valores_estatus = cursor.fetchall()
for fila in valores_estatus:
    print(fila)

------ Atributos de la tabla -------
Nombre: id_estatus, Tipo: tinyint(4), Nulo: NO, Clave: PRI, Predeterminado: None, Extra: 
Nombre: estatus, Tipo: varchar(100), Nulo: NO, Clave: , Predeterminado: None, Extra: 
------------------------------------
Valores de la tabla estatus
(1, 'En licencia')
(2, 'Activo')
(3, 'Jubilado')
(4, 'Intercambio')
(5, 'Articulo 33')
(6, 'Articulo 34')
(7, 'Articulo 35')
(8, 'Baja Voluntaria')
(9, 'Baja Administrativa')
(10, 'Desactivado')


Se comparten 10 estatus, aplicables para estudiantes y profesores de la universidad, que van desde en licencia hasta desactivado.

* **En licencia**: El maestro o estudiante ha enviado a la coordinación de la carrera una solicitud para realizar otras actividades y retomar el plan de estudios o cargo más adelante.
* **Activo**: El estudiante o profesor se encuentran activos para interactuar en la aplicación Web.
* **Jubilado**: El maestro ha terminado su etapa como docente en la universidad después de laborar por un tipo determinado, avalado por la junta administrativa.
* **Intercambio**: El maestro o estudiante realiza un intercambio en otro país.
* **Artículo 33**: Si por cualquier circunstancia el estudiante no aprueba en ordinario, ni extraordinario una materia, tienes derecho a repetirla en el ciclo escolar inmediato siguiente, teniendo la oportunidad de aprobarla en ordinario o extraordinario, si en estas oportunidades no la aprueba, se le dará de baja de la Universidad.
* **Artículo 34**: El estudiante que haya sido dado de baja conforme al artículo 33 podrá solicitar por escrito a la Comisión de Educación del Consejo de Centro o de Escuela, antes del inicio del ciclo inmediato siguiente en que haya sido dado de baja, una nueva oportunidad para acreditar la materia o materias que adeude.
* **Artículo 35**: A los estudiantes que sean dados de baja de la Universidad conforme a los artículos 32, 33 y 34 de este ordenamiento, no se les autorizará su reingreso a la carrera o posgrado por el cual se les dio de baja.
* **Baja Voluntaria**: El estudiante o el profesor dejan sus responsabilidades de la universidad de manera voluntaria.
* **Baja Administrativa**: Haber infringido alguna de las obligaciones o reglamentos institucionales.
* **Desactivado**: El estudiante o profesor no se encuentran activos para interactuar en la aplicación Web.

**Revisamos los atributos e información que componen la tabla estudiantes**


In [10]:
# Revisamos como esta compuesta la información de la tabla estudiantes
cursor.execute('DESCRIBE estudiantes;')
# Obtener resultados
estudiantes = cursor.fetchall()
print("------ Atributos de la tabla -------")
for atributo in estudiantes:
    print(f"Nombre: {atributo[0]}, Tipo: {atributo[1]}, Nulo: {atributo[2]}, Clave: {atributo[3]}, Predeterminado: {atributo[4]}, Extra: {atributo[5]}")
print("------------------------------------")


#Imprimimos los valores de la tabla estudiantes
print("Valores de la tabla estudiantes")
cursor.execute('SELECT * FROM estudiantes;')
# Obtener resultados
valores_estudiantes = cursor.fetchall()
for fila in valores_estudiantes:
    print(fila)

------ Atributos de la tabla -------
Nombre: codigo_estudiante, Tipo: varchar(50), Nulo: NO, Clave: PRI, Predeterminado: None, Extra: 
Nombre: nombres, Tipo: varchar(100), Nulo: NO, Clave: , Predeterminado: None, Extra: 
Nombre: apellido_paterno, Tipo: varchar(100), Nulo: NO, Clave: , Predeterminado: None, Extra: 
Nombre: apellido_materno, Tipo: varchar(100), Nulo: NO, Clave: , Predeterminado: None, Extra: 
Nombre: nivel, Tipo: tinyint(4), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
Nombre: situacion, Tipo: tinyint(4), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
Nombre: ultimo_ciclo, Tipo: tinyint(4), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
Nombre: ciclo_admision, Tipo: tinyint(4), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
Nombre: genero, Tipo: tinyint(4), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
Nombre: carrera, Tipo: tinyint(4), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
Nombre: centro_universitario, Tipo: tinyint(4), Nulo: NO

En esta universidad se encuentra el registro de 22 estudiantes hasta este momento.

**Revisamos los atributos e información que componen la tabla generos**

In [11]:
# Revisamos como esta compuesta la información de la tabla generos
cursor.execute('DESCRIBE generos;')
# Obtener resultados
generos = cursor.fetchall()
print("------ Atributos de la tabla -------")
for atributo in generos:
    print(f"Nombre: {atributo[0]}, Tipo: {atributo[1]}, Nulo: {atributo[2]}, Clave: {atributo[3]}, Predeterminado: {atributo[4]}, Extra: {atributo[5]}")
print("------------------------------------")


#Imprimimos los valores de la tabla generos
print("Valores de la tabla generos")
cursor.execute('SELECT * FROM generos;')
# Obtener resultados
valores_generos = cursor.fetchall()
for fila in valores_generos:
    print(fila)

------ Atributos de la tabla -------
Nombre: id_genero, Tipo: tinyint(4), Nulo: NO, Clave: PRI, Predeterminado: None, Extra: 
Nombre: genero, Tipo: varchar(100), Nulo: NO, Clave: , Predeterminado: None, Extra: 
------------------------------------
Valores de la tabla generos
(1, 'Masculino')
(2, 'Femenino')


Se disponen de dos géneros: Masculino y femenino para la identificación de profesores y estudiantes.

**Revisamos los atributos e información que componen la tabla gestores**

In [12]:
# Revisamos como esta compuesta la información de la tabla carreras
cursor.execute('DESCRIBE gestores;')
# Obtener resultados
gestores = cursor.fetchall()
print("------ Atributos de la tabla -------")
for atributo in gestores:
    print(f"Nombre: {atributo[0]}, Tipo: {atributo[1]}, Nulo: {atributo[2]}, Clave: {atributo[3]}, Predeterminado: {atributo[4]}, Extra: {atributo[5]}")
print("------------------------------------")


#Imprimimos los valores de la tabla gestores
print("Valores de la tabla gestores")
cursor.execute('SELECT * FROM gestores;')
# Obtener resultados
valores_gestores = cursor.fetchall()
for fila in valores_gestores:
    print(fila)

------ Atributos de la tabla -------
Nombre: id_departamento, Tipo: tinyint(4), Nulo: NO, Clave: PRI, Predeterminado: None, Extra: 
Nombre: departamento, Tipo: varchar(50), Nulo: NO, Clave: , Predeterminado: None, Extra: 
Nombre: division, Tipo: varchar(10), Nulo: NO, Clave: , Predeterminado: None, Extra: 
Nombre: cordinador_carrera, Tipo: varchar(50), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
------------------------------------
Valores de la tabla gestores
(1, 'Administración', 'Admon', '1111111')
(2, 'Economía', 'Economía', '1111117')


Se cuenta con 2 registros con relación a los gestores de la información.

**Revisamos los atributos e información que componen la tabla nacionalidades**

In [13]:
# Revisamos como esta compuesta la información de la tabla nacionalidades
cursor.execute('DESCRIBE nacionalidades;')
# Obtener resultados
nacionalidades = cursor.fetchall()
print("------ Atributos de la tabla -------")
for atributo in nacionalidades:
    print(f"Nombre: {atributo[0]}, Tipo: {atributo[1]}, Nulo: {atributo[2]}, Clave: {atributo[3]}, Predeterminado: {atributo[4]}, Extra: {atributo[5]}")
print("------------------------------------")


#Imprimimos los valores de la tabla carreras
print("Valores de la tabla nacionalidades")
cursor.execute('SELECT * FROM nacionalidades;')
# Obtener resultados
valores_nacionalidades = cursor.fetchall()
for fila in valores_nacionalidades:
    print(fila)

------ Atributos de la tabla -------
Nombre: id_nacionalidad, Tipo: tinyint(4), Nulo: NO, Clave: PRI, Predeterminado: None, Extra: 
Nombre: nacionalidad, Tipo: varchar(100), Nulo: NO, Clave: , Predeterminado: None, Extra: 
------------------------------------
Valores de la tabla nacionalidades
(1, 'Argentina')
(2, 'Americana')
(3, 'Boliviana')
(4, 'Chilena')
(5, 'Colombiana')
(6, 'Costarricense')
(7, 'Cubana')
(8, 'Ecuatoriana')
(9, 'Española')
(10, 'Francesa')
(11, 'Guatemalteca')
(12, 'Hondureña')
(13, 'Mexicana')
(14, 'Nicaragüense')
(15, 'Panameña')
(16, 'Paraguaya')
(17, 'Peruana')
(18, 'Dominicana')
(19, 'Uruguaya')
(20, 'Venezolana')


Tenemos 20 nacionalidades para los profesores que nos visitan. Gracias a este análisis se discute la posibilidad de crear una relación y campo para la tabla de estudiantes, ya que se cuenta con la presencia de estudiantes de intercambio provenientes de otros países.

**Revisamos los atributos e información que componen la tabla niveles_educativos**

In [14]:
# Revisamos como esta compuesta la información de la tabla niveles_educativos
cursor.execute('DESCRIBE niveles_educativos;')
# Obtener resultados
niveles_educativos = cursor.fetchall()
print("------ Atributos de la tabla -------")
for atributo in niveles_educativos:
    print(f"Nombre: {atributo[0]}, Tipo: {atributo[1]}, Nulo: {atributo[2]}, Clave: {atributo[3]}, Predeterminado: {atributo[4]}, Extra: {atributo[5]}")
print("------------------------------------")


#Imprimimos los valores de la tabla niveles_educativos
print("Valores de la tabla niveles_educativos")
cursor.execute('SELECT * FROM niveles_educativos;')
# Obtener resultados
valores_niveles_educativos = cursor.fetchall()
for fila in valores_niveles_educativos:
    print(fila)

------ Atributos de la tabla -------
Nombre: id_nivel, Tipo: tinyint(4), Nulo: NO, Clave: PRI, Predeterminado: None, Extra: 
Nombre: nivel, Tipo: varchar(100), Nulo: NO, Clave: , Predeterminado: None, Extra: 
------------------------------------
Valores de la tabla niveles_educativos
(1, 'Bachillerato')
(2, 'Técnico Superior Universitario')
(3, 'Licenciatura')
(4, 'Maestría')
(5, 'Doctorado')
(6, 'Post Doctorado')
(7, 'Especialidad')
(8, 'Sub-Especialidad')


Esta tabla hace referencia al grado de estudios alcanzado por el estudiante, se discute crear la llave foránea para profesores.

**Revisamos los atributos e información que componen la tabla nrc**

In [15]:
# Revisamos como esta compuesta la información de la tabla nrc
cursor.execute('DESCRIBE nrc;')
# Obtener resultados
nrc = cursor.fetchall()
print("------ Atributos de la tabla -------")
for atributo in nrc:
    print(f"Nombre: {atributo[0]}, Tipo: {atributo[1]}, Nulo: {atributo[2]}, Clave: {atributo[3]}, Predeterminado: {atributo[4]}, Extra: {atributo[5]}")
print("------------------------------------")


#Imprimimos los valores de la tabla nrc
print("Valores de la tabla nrc")
cursor.execute('SELECT * FROM nrc;')
# Obtener resultados
valores_nrc = cursor.fetchall()
for fila in valores_nrc:
    print(fila)

------ Atributos de la tabla -------
Nombre: id_nrc, Tipo: tinyint(4), Nulo: NO, Clave: , Predeterminado: None, Extra: 
Nombre: nrc, Tipo: varchar(50), Nulo: NO, Clave: PRI, Predeterminado: None, Extra: 
Nombre: codigo_profesor, Tipo: varchar(50), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
Nombre: materia, Tipo: varchar(100), Nulo: NO, Clave: , Predeterminado: None, Extra: 
Nombre: departamento, Tipo: tinyint(4), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
Nombre: seccion, Tipo: varchar(50), Nulo: NO, Clave: , Predeterminado: None, Extra: 
Nombre: ciclo, Tipo: tinyint(4), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
------------------------------------
Valores de la tabla nrc
(1, 'NRC001', '1111111', 'Matemáticas I', 2, 'A', 1)
(2, 'NRC002', '1111112', 'Contabilidad I', 2, 'B', 1)
(3, 'NRC003', '1111113', 'Contabilidad II', 5, 'A', 1)
(4, 'NRC004', '1111114', 'Administración', 4, 'C', 1)
(5, 'NRC005', '1111115', 'Marketing', 3, 'D', 1)
(6, 'NRC006', '1111116', '

Hasta este momento se cuentan con 10 materias dadas de alta, asignadas a 10 profesores.

**Revisamos los atributos e información que componen la tabla preguntas**

In [16]:
# Revisamos como esta compuesta la información de la tabla preguntas
cursor.execute('DESCRIBE preguntas;')
# Obtener resultados
preguntas = cursor.fetchall()
print("------ Atributos de la tabla -------")
for atributo in preguntas:
    print(f"Nombre: {atributo[0]}, Tipo: {atributo[1]}, Nulo: {atributo[2]}, Clave: {atributo[3]}, Predeterminado: {atributo[4]}, Extra: {atributo[5]}")
print("------------------------------------")


#Imprimimos los valores de la tabla preguntas
print("Valores de la tabla preguntas")
cursor.execute('SELECT * FROM preguntas;')
# Obtener resultados
valores_preguntas = cursor.fetchall()
for fila in valores_preguntas:
    print(fila)

------ Atributos de la tabla -------
Nombre: id_pregunta, Tipo: tinyint(4), Nulo: NO, Clave: PRI, Predeterminado: None, Extra: 
Nombre: pregunta, Tipo: varchar(100), Nulo: NO, Clave: , Predeterminado: None, Extra: 
Nombre: ciclo_escolar, Tipo: tinyint(4), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
------------------------------------
Valores de la tabla preguntas
(1, 'Puntualidad', 1)
(2, 'Respeto', 1)
(3, 'Educación', 1)
(4, 'Metodología de enseñanza', 1)
(5, 'Conocimiento del contenido', 1)
(6, 'Comunicación', 1)
(7, 'Adaptabilidad', 1)
(8, 'Retroalimentación', 1)
(9, 'Manejo del aula', 1)
(10, 'Innovación y creatividad', 1)


En esta tabla se almacenan las preguntas de esta encuesta, **Nota: Los valores que se comprenden como pregunta son las categorías que pueden ser sustituidas a preguntas posteriormente**:

* **Pregunta 1**: Hace referencia a una pregunta enfocada en la puntualidad del docente.
* **Pregunta 2**: Se enfoca la pregunta hacia el respeto del docente hacia sus estudiantes.
* **Pregunta 3**: La importancia de la educación que recibió el estudiante.
* **Pregunta 4**: El método de enseñanza fue de utilidad para el estudiante.
* **Pregunta 5**: El conocimiento compartido fue de utilidad para el estudiante.
* **Pregunta 6**: La comunicación fue asertiva.
* **Pregunta 7**: El profesor se adaptó a las necesidades de los estudiantes.
* **Pregunta 8**: Se le dio retroalimentación a los estudiantes sobre los trabajos entregados.
* **Pregunta 9**: El profesor supo manejar el aula.
* **Pregunta 10**: El profesor utilizó recursos tecnológicos en el curso.

**Revisamos los atributos e información que componen la tabla profesores**

In [17]:
# Revisamos como esta compuesta la información de la tabla profesores
cursor.execute('DESCRIBE profesores;')
# Obtener resultados
profesores = cursor.fetchall()
print("------ Atributos de la tabla -------")
for atributo in profesores:
    print(f"Nombre: {atributo[0]}, Tipo: {atributo[1]}, Nulo: {atributo[2]}, Clave: {atributo[3]}, Predeterminado: {atributo[4]}, Extra: {atributo[5]}")
print("------------------------------------")


#Imprimimos los valores de la tabla profesores
print("Valores de la tabla profesores")
cursor.execute('SELECT * FROM profesores;')
# Obtener resultados
valores_profesores = cursor.fetchall()
for fila in valores_profesores:
    print(fila)

------ Atributos de la tabla -------
Nombre: codigo_profesor, Tipo: varchar(50), Nulo: NO, Clave: PRI, Predeterminado: None, Extra: 
Nombre: nombres, Tipo: varchar(100), Nulo: NO, Clave: , Predeterminado: None, Extra: 
Nombre: apellido_paterno, Tipo: varchar(100), Nulo: NO, Clave: , Predeterminado: None, Extra: 
Nombre: apellido_materno, Tipo: varchar(100), Nulo: NO, Clave: , Predeterminado: None, Extra: 
Nombre: genero, Tipo: tinyint(4), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
Nombre: grado_estudio, Tipo: varchar(100), Nulo: NO, Clave: , Predeterminado: None, Extra: 
Nombre: SNI, Tipo: varchar(100), Nulo: NO, Clave: , Predeterminado: None, Extra: 
Nombre: departamento_adscripcion, Tipo: tinyint(4), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
Nombre: centro_pertenencia, Tipo: tinyint(4), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
Nombre: tipo_contrato, Tipo: tinyint(4), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
Nombre: horas_semanales, Tipo: int(4

Hasta este momento se cuenta con una plantilla de 10 profesores en la universidad, todos con el estatus activo.

**Revisamos los atributos e información que componen la tabla respuesta_preguntas**

In [18]:
# Revisamos como esta compuesta la información de la tabla respuesta_preguntas
cursor.execute('DESCRIBE respuesta_preguntas;')
# Obtener resultados
respuesta_preguntas = cursor.fetchall()
print("------ Atributos de la tabla -------")
for atributo in respuesta_preguntas:
    print(f"Nombre: {atributo[0]}, Tipo: {atributo[1]}, Nulo: {atributo[2]}, Clave: {atributo[3]}, Predeterminado: {atributo[4]}, Extra: {atributo[5]}")
print("------------------------------------")


#Imprimimos los valores de la tabla respuesta_preguntas
print("Valores de la tabla respuesta_preguntas")
cursor.execute('SELECT * FROM respuesta_preguntas;')
# Obtener resultados
valores_respuesta_preguntas = cursor.fetchall()
for fila in valores_respuesta_preguntas:
    print(fila)

------ Atributos de la tabla -------
Nombre: id_respuesta, Tipo: tinyint(4), Nulo: NO, Clave: PRI, Predeterminado: None, Extra: 
Nombre: respuesta, Tipo: varchar(100), Nulo: NO, Clave: , Predeterminado: None, Extra: 
Nombre: ciclo_escolar, Tipo: tinyint(4), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
------------------------------------
Valores de la tabla respuesta_preguntas
(1, 'Totalmente de acuerdo', 1)
(2, 'De acuerdo', 1)
(3, 'En desacuerdo', 1)
(4, 'Totalmente en desacuerdo', 1)


Se tienen las siguientes respuestas, relacionadas con las preguntas de la encuesta del ciclo 2024A:

* **Totalmente de acuerdo**
* **De acuerdo**
* **En desacuerdo**
* **Totalmente de desacuerdo**



**Revisamos los atributos e información que componen la tabla resultado_encuesta_historico**

In [19]:
# Revisamos como esta compuesta la información de la tabla resultado_encuesta_historico
cursor.execute('DESCRIBE resultado_encuesta_historico;')
# Obtener resultados
resultado_encuesta_historico = cursor.fetchall()
print("------ Atributos de la tabla -------")
for atributo in resultado_encuesta_historico:
    print(f"Nombre: {atributo[0]}, Tipo: {atributo[1]}, Nulo: {atributo[2]}, Clave: {atributo[3]}, Predeterminado: {atributo[4]}, Extra: {atributo[5]}")
print("------------------------------------")


#Imprimimos los valores de la tabla resultado_encuesta_historico
print("Valores de la tabla resultado_encuesta_historico")
cursor.execute('SELECT * FROM resultado_encuesta_historico;')
# Obtener resultados
valores_resultado_encuesta_historico = cursor.fetchall()
for fila in valores_resultado_encuesta_historico:
    print(fila)

------ Atributos de la tabla -------
Nombre: codigo_profesor, Tipo: varchar(50), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
Nombre: codigo_estudiante, Tipo: varchar(100), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
Nombre: encuesta, Tipo: tinyint(4), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
Nombre: ciclo, Tipo: tinyint(4), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
Nombre: no_pregunta, Tipo: tinyint(4), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
Nombre: nrc, Tipo: varchar(50), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
Nombre: id_respuesta, Tipo: tinyint(4), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
Nombre: puntaje, Tipo: tinyint(4), Nulo: NO, Clave: , Predeterminado: None, Extra: 
------------------------------------
Valores de la tabla resultado_encuesta_historico
('1111111', '111111111', 1, 1, 4, 'NRC001', 4, 0)
('1111111', '111111111', 1, 1, 5, 'NRC001', 1, 2)
('1111111', '111111111', 1, 1, 6, 'NRC001', 2, 1)
('111111

En esta tabla se almacena el historial de respuestas emitidas por los estudiantes conforme a las encuestas.

**Revisamos los atributos e información que componen la tabla resultado_encuesta_ultimo_ciclo**

In [20]:
# Revisamos como esta compuesta la información de la tabla resultado_encuesta_ultimo_ciclo
cursor.execute('DESCRIBE resultado_encuesta_ultimo_ciclo;')
# Obtener resultados
resultado_encuesta_ultimo_ciclo = cursor.fetchall()
print("------ Atributos de la tabla -------")
for atributo in resultado_encuesta_ultimo_ciclo:
    print(f"Nombre: {atributo[0]}, Tipo: {atributo[1]}, Nulo: {atributo[2]}, Clave: {atributo[3]}, Predeterminado: {atributo[4]}, Extra: {atributo[5]}")
print("------------------------------------")


#Imprimimos los valores de la tabla resultado_encuesta_ultimo_ciclo
print("Valores de la tabla resultado_encuesta_ultimo_ciclo")
cursor.execute('SELECT * FROM resultado_encuesta_ultimo_ciclo;')
# Obtener resultados
valores_resultado_encuesta_ultimo_ciclo = cursor.fetchall()
for fila in valores_resultado_encuesta_ultimo_ciclo:
    print(fila)

------ Atributos de la tabla -------
Nombre: codigo_profesor, Tipo: varchar(50), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
Nombre: codigo_estudiante, Tipo: varchar(100), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
Nombre: encuesta, Tipo: tinyint(4), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
Nombre: ciclo, Tipo: tinyint(4), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
Nombre: no_pregunta, Tipo: tinyint(4), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
Nombre: nrc, Tipo: varchar(50), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
Nombre: id_respuesta, Tipo: tinyint(4), Nulo: NO, Clave: MUL, Predeterminado: None, Extra: 
Nombre: puntaje, Tipo: tinyint(4), Nulo: YES, Clave: , Predeterminado: None, Extra: 
------------------------------------
Valores de la tabla resultado_encuesta_ultimo_ciclo
('1111111', '111111111', 1, 1, 4, 'NRC001', 4, 0)
('1111111', '111111111', 1, 1, 5, 'NRC001', 1, 2)
('1111111', '111111111', 1, 1, 6, 'NRC001', 2, 1)
('11

En esta tabla se almacenan las respuestas de los estudiantes del ciclo actual, teniendo hasta este momento 295 respuestas emitidas por los estudiantes.

**Revisamos los atributos e información que componen la tabla roles**

In [21]:
# Revisamos como esta compuesta la información de la tabla preguntas
cursor.execute('DESCRIBE roles;')
# Obtener resultados
roles = cursor.fetchall()
print("------ Atributos de la tabla -------")
for atributo in roles:
    print(f"Nombre: {atributo[0]}, Tipo: {atributo[1]}, Nulo: {atributo[2]}, Clave: {atributo[3]}, Predeterminado: {atributo[4]}, Extra: {atributo[5]}")
print("------------------------------------")


#Imprimimos los valores de la tabla roles
print("Valores de la tabla roles")
cursor.execute('SELECT * FROM roles;')
# Obtener resultados
valores_roles = cursor.fetchall()
for fila in valores_roles:
    print(fila)

------ Atributos de la tabla -------
Nombre: id_rol, Tipo: tinyint(4), Nulo: NO, Clave: PRI, Predeterminado: None, Extra: 
Nombre: rol, Tipo: varchar(100), Nulo: NO, Clave: , Predeterminado: None, Extra: 
------------------------------------
Valores de la tabla roles
(1, 'Administración y mantenimiento del sistema')
(2, 'Gestión de la información')
(3, 'Rectoria')
(4, 'Secretario Académico')
(5, 'Directores de División')
(6, 'Jefes de departamento')
(7, 'Coordinadores de Carrera')
(8, 'Profesores')
(9, 'Estudiantes')


Se cuentan con 9 roles para la Aplicación Web:

* **Administración y mantenimiento del sistema**: Se encargan de administrar y darle mantenimiento a la Aplicación Web.
* **Gestión de la información**: Gestionan la información a los diferentes departamentos de la universidad.
* **Rectoría**: Visualizan la información de los centros universitarios, mediante dashboards.
* **Secretario Académico**: Recibe los informes de gestión de la información y visualiza la información de los centros universitarios mediante dashboards.
* **Directores de División**: Visualizan las estadísticas de la encuesta con base en su división.
* **Jefes de departamento**: Visualizan las estadísticas de la encuesta con base en sus departamentos.
* **Coordinadores de Carrera**: Analizan la información obtenida por sus respectivas carreras.
* **Profesores**: Visualizan sus datos con forma a la aplicación de la encuesta.
* **Estudiantes**: Contestan la encuesta con base en su ciclo en curso.

**Revisamos los atributos e información que componen la tabla situaciones_academicas**

In [22]:
# Revisamos como esta compuesta la información de la tabla situaciones_academicas
cursor.execute('DESCRIBE situaciones_academicas;')
# Obtener resultados
situaciones_academicas = cursor.fetchall()
print("------ Atributos de la tabla -------")
for atributo in situaciones_academicas:
    print(f"Nombre: {atributo[0]}, Tipo: {atributo[1]}, Nulo: {atributo[2]}, Clave: {atributo[3]}, Predeterminado: {atributo[4]}, Extra: {atributo[5]}")
print("------------------------------------")


#Imprimimos los valores de la tabla situaciones_academicas
print("Valores de la tabla situaciones_academicas")
cursor.execute('SELECT * FROM situaciones_academicas;')
# Obtener resultados
valores_situaciones_academicas = cursor.fetchall()
for fila in valores_situaciones_academicas:
    print(fila)

------ Atributos de la tabla -------
Nombre: id_situacion_academica, Tipo: tinyint(4), Nulo: NO, Clave: PRI, Predeterminado: None, Extra: 
Nombre: situacion_academica, Tipo: varchar(100), Nulo: NO, Clave: , Predeterminado: None, Extra: 
------------------------------------
Valores de la tabla situaciones_academicas
(1, 'En licencia')
(2, 'Activo')
(3, 'Intercambio')
(4, 'Articulo 33')
(5, 'Articulo 34')
(6, 'Articulo 35')
(7, 'Baja Voluntaria')
(8, 'Baja Administrativa')
(9, 'Desactivado')


Se tienen 9 situaciones académicas, aplicables solo para estudiantes de la universidad, que van desde en licencia hasta desactivado.

* **En licencia**: El estudiante ha enviado a la coordinación de la carrera una solicitud para realizar otras actividades y retomar el plan de estudios o cargo más adelante.
* **Activo**: El estudiante se encuentran activos para interactuar en la aplicación Web.
* **Intercambio**: El estudiante realizan un intercambio en otro país.
* **Artículo 33**: Si por cualquier circunstancia el estudiante no aprueba en ordinario, ni extraordinario una materia, tienes derecho a repetirla en el ciclo escolar inmediato siguiente, teniendo la oportunidad de aprobarla en ordinario o extraordinario, si en estas oportunidades no la aprueba, se le dará de baja de la Universidad.
* **Artículo 34**: El estudiante que haya sido dado de baja conforme al artículo 33 podrá solicitar por escrito a la Comisión de Educación del Consejo de Centro o de Escuela, antes del inicio del ciclo inmediato siguiente en que haya sido dado de baja, una nueva oportunidad para acreditar la materia o materias que adeude.
* **Artículo 35**: A los estudiantes que sean dados de baja de la Universidad conforme a los artículos 32, 33 y 34 de este ordenamiento, no se les autorizará su reingreso a la carrera o posgrado por el cual se les dio de baja.
* **Baja Voluntaria**: El estudiante o el profesor dejan sus responsabilidades de la universidad de manera voluntaria.
* **Baja Administrativa**: Haber infringido alguna de las obligaciones o reglamentos institucionales.
* **Desactivado**: El estudiante no se encuentra activo para interactuar en la aplicación Web.

**Revisamos los atributos e información que componen la tabla tipos_contratos**

In [23]:
# Revisamos como esta compuesta la información de la tabla tipos_contratos
cursor.execute('DESCRIBE tipos_contratos;')
# Obtener resultados
tipos_contratos = cursor.fetchall()
print("------ Atributos de la tabla -------")
for atributo in tipos_contratos:
    print(f"Nombre: {atributo[0]}, Tipo: {atributo[1]}, Nulo: {atributo[2]}, Clave: {atributo[3]}, Predeterminado: {atributo[4]}, Extra: {atributo[5]}")
print("------------------------------------")


#Imprimimos los valores de la tabla tipos_contratos
print("Valores de la tabla tipos_contratos")
cursor.execute('SELECT * FROM tipos_contratos;')
# Obtener resultados
valores_tipos_contratos = cursor.fetchall()
for fila in valores_tipos_contratos:
    print(fila)

------ Atributos de la tabla -------
Nombre: id_tipo_contrato, Tipo: tinyint(4), Nulo: NO, Clave: PRI, Predeterminado: None, Extra: 
Nombre: tipo_contrato, Tipo: varchar(100), Nulo: NO, Clave: , Predeterminado: None, Extra: 
------------------------------------
Valores de la tabla tipos_contratos
(1, 'Definitivo')
(2, 'Permanente o por tiempo indeterminado')
(3, 'Permanente o por tiempo indeterminado')


Se contemplan los tres tipos de contratos que pueden tener profesores en la universidad:

* **Definivo**: El profesor es un profesor de planta en la universidad.
* **Permanente o por tiempo indeterminado**: El profesor realizará sus labores por tiempo permanente.
* **Por tiempo definido o determinado**: El profesor realizará sus labores por un tiempo determinado.

---

# <a id="3">3. Limpieza de datos<a> 🛠

En esta etapa vamos a introducir o estandarizar la información para el análisis posterior, con base a lo presentado en la etapa 2, descripción de los datos. Se tiene que la tabla tipo de contrato, cuenta con un valor, el cual no empieza con una letra mayúscula, para lo cual vamos a corregirlo.

In [24]:
# Ejecutamos y actualizamos el registro con base al id del tipo de contrato
cursor.execute('UPDATE tipos_contratos SET tipo_contrato = "Permanente o por tiempo indeterminado" WHERE id_tipo_contrato = 2;')
#Realizamos el commit
conexion.commit()


#Imprimimos los valores de la tabla tipos_contratos
print("Valores de la tabla tipos_contratos")
cursor.execute('SELECT * FROM tipos_contratos;')
# Obtener resultados
valores_tipos_contratos = cursor.fetchall()
for fila in valores_tipos_contratos:
    print(fila)

Valores de la tabla tipos_contratos
(1, 'Definitivo')
(2, 'Permanente o por tiempo indeterminado')
(3, 'Permanente o por tiempo indeterminado')


---

# <a id="4">4. Missing values<a> 🔮

De acuerdo con lo realizado en la etapa 2, descripción de los datos. Algunos estudiantes no presentan el nombre materno, lo cual vamos a corregir, poniendo el valor de **Sin Información**.

In [25]:
# Ejecutamos y actualizamos el registro con base a los ids de los estudiantes
cursor.execute('UPDATE estudiantes SET apellido_materno = "Sin Información" WHERE codigo_estudiante IN (111111012,111111015,111111017,111111019,111111021,111111117);')
#Realizamos el commit
conexion.commit()


#Imprimimos los valores de la tabla estudiantes
print("Valores de la tabla estudiantes")
cursor.execute('SELECT * FROM estudiantes;')
# Obtener resultados
valores_estudiantes = cursor.fetchall()
for fila in valores_estudiantes:
    print(fila)

Valores de la tabla estudiantes
('111111011', 'estudiante_011', 'eappelido_pat011', 'eappelido_mat011', 2, 4, 1, 1, 1, 1, 3, 2, 9, '$2b$12$rln.OhhvbnJurBGsPQg.OO6/jbDVUbFF/mSG/l6nhlf/.8gul4UKe', 0)
('111111012', 'estudiante_012', 'eappelido_pat012', 'Sin Información', 1, 2, 1, 1, 2, 1, 3, 2, 9, '$2b$12$rln.OhhvbnJurBGsPQg.OO6/jbDVUbFF/mSG/l6nhlf/.8gul4UKe', 0)
('111111013', 'estudiante_013', 'eappelido_pat013', 'eappelido_mat013', 4, 3, 1, 1, 1, 1, 3, 2, 9, '$2b$12$rln.OhhvbnJurBGsPQg.OO6/jbDVUbFF/mSG/l6nhlf/.8gul4UKe', 0)
('111111014', 'estudiante_014', 'eappelido_pat014', 'eappelido_mat014', 2, 5, 1, 1, 2, 1, 3, 2, 9, '$2b$12$rln.OhhvbnJurBGsPQg.OO6/jbDVUbFF/mSG/l6nhlf/.8gul4UKe', 0)
('111111015', 'estudiante_015', 'eappelido_pat015', 'Sin Información', 3, 4, 1, 1, 1, 1, 3, 2, 9, '$2b$12$rln.OhhvbnJurBGsPQg.OO6/jbDVUbFF/mSG/l6nhlf/.8gul4UKe', 0)
('111111016', 'estudiante_016', 'eappelido_pat016', 'eappelido_mat016', 1, 2, 1, 1, 2, 1, 3, 2, 9, '$2b$12$rln.OhhvbnJurBGsPQg.OO6/jbDVUbFF/

---

# <a id="5">5. Visualización de datos<a> 📊

Para el análisis y visualización de la base de datos limpia, nos centraremos en las tablas, las cuales presentarán registros y no son catalogadas como catálogos, por ejemplo: Estudiantes,
profesores, resultado_encuesta_historico y resultado_encuesta_ultimo_ciclo.

## ¿Cuántos estudiantes tenemos en cada nivel?

In [26]:
# Consulta SQL (estudiantes - nivel educativo)
query = 'SELECT ne.nivel AS Nivel, COUNT(ne.nivel) AS Nivel_Educativo FROM estudiantes e INNER JOIN niveles_educativos ne ON e.nivel = ne.id_nivel GROUP BY ne.nivel'

# Leemos la query y se la asignamos a un dataframe
df_niveles_educativos = pd.read_sql(query, conexion)

#Generamos la gráfica
alt.Chart(df_niveles_educativos).mark_bar().encode(
    alt.X("Nivel:N", title="Número de estudiantes por nivel"),
    alt.Y("Nivel_Educativo:Q", title="Nivel Educativo"),
    alt.Color("Nivel:N", title="Número de estudiantes por nivel"),
).properties(
    width=1500,
    height=500,
    title="¿Cuántos estudiantes tenemos en cada nivel?"
)


<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


## ¿Cuántos estudiantes tienen una situación academica?

In [27]:
# Consulta SQL (estudiantes - Situacion academica)
query = 'SELECT sa.situacion_academica, COUNT(sa.situacion_academica) AS count FROM estudiantes e INNER JOIN situaciones_academicas sa ON e.situacion = sa.id_situacion_academica GROUP BY sa.situacion_academica'

# Ejecutamos la consulta y asignamos la infromación a un dataframe
df_situacion_academica = pd.read_sql(query, conexion)

#Graficamos la infromación
alt.Chart(df_situacion_academica).mark_bar().encode(
    alt.X("situacion_academica:N", title="Situaciónes academicas"),
    alt.Y("count:Q", title="Número de situaciónes acedemicas por estudiante"),
    alt.Color("situacion_academica:N", title="Situaciónes academicas"),
).properties(
    width=1500,
    height=500,
    title="¿Cuántos estudiantes tienen una situación academica?"
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


## ¿Cuántos hombres y mujeres estudian en la universidad?

In [28]:
# Generamos la consulta de estudiantes - generos
query = 'SELECT  g.genero, COUNT(g.genero) AS count  FROM estudiantes e INNER JOIN generos g ON e.genero = g.id_genero GROUP BY g.genero'

# Asignamos los resultados al dataframe
df_genero = pd.read_sql(query, conexion)

#Graficamos la infromación
alt.Chart(df_genero).mark_bar().encode(
    alt.X("genero:N", title="Géneros"),
    alt.Y("count:Q", title="Número de estudiantes Femenino y Masculino"),
    alt.Color("genero:N", title="Géneros"),
).properties(
    width=1500,
    height=500,
    title="¿Cuántos hombres y mujeres estudian en la universidad?"
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


## ¿Cuántos profesores tienen un grado de SNII?

In [29]:
# Generamos la consulta de profesores - snii
query = 'SELECT  p.sni, COUNT(p.sni) AS count FROM profesores p GROUP BY p.sni'

# Asignamos los resultados al dataframe
df_snii = pd.read_sql(query, conexion)

#Graficamos la infromación
alt.Chart(df_snii).mark_bar().encode(
    alt.X("sni:N", title="Grado SNII"),
    alt.Y("count:Q", title="Número de SNII por profesor"),
    alt.Color("sni:N", title="Grado SNII"),
).properties(
    width=1500,
    height=500,
    title="¿Cuántos profesores tienen un grado de SNII?"
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


## ¿Cuántos profesores, hombres y mujeres tenemos en la universidad?

In [30]:
# Generamos la consulta de estudiantes - generos
query = 'SELECT  g.genero, COUNT(g.genero) AS count FROM profesores p INNER JOIN generos g ON p.genero = g.id_genero GROUP BY g.genero'

# Asignamos los resultados al dataframe
df_profesor_genero = pd.read_sql(query, conexion)

#Graficamos la infromación
alt.Chart(df_profesor_genero).mark_bar().encode(
    alt.X("genero:N", title="Géneros"),
    alt.Y("count:Q", title="Número de profesores Femenino y Masculino"),
    alt.Color("genero:N", title="Géneros"),
).properties(
    width=1500,
    height=500,
    title="¿Cuántos profesores, hombres y mujeres tenemos en la universidad?"
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


## ¿Cuántos profesores cuentan con un grado educativo?

In [31]:
# Consulta SQL (profesores - nivel educativo)
query = 'SELECT ne.nivel AS Nivel, COUNT(ne.nivel) AS Nivel_Educativo FROM profesores p INNER JOIN niveles_educativos ne ON p.grado_estudio = ne.id_nivel GROUP BY ne.nivel'

# Leemos la query y se la asignamos a un dataframe
df_profesor_nivele_educativo = pd.read_sql(query, conexion)

#Generamos la gráfica
alt.Chart(df_profesor_nivele_educativo).mark_bar().encode(
    alt.X("Nivel:N", title="Nivel educativo alcanzado"),
    alt.Y("Nivel_Educativo:Q", title="Número de profesores por nivel"),
    alt.Color("Nivel:N", title="Nivel educativo alcanzado"),
).properties(
    width=1500,
    height=500,
    title="¿Cuántos profesores cuentan con un grado educativo?"
)


<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


## ¿Cuántos profesores tenemos por tipo de contrato?

In [32]:
# Consulta SQL (profesores - tipo de contrato)
query = 'SELECT tp.tipo_contrato, COUNT(tp.tipo_contrato) AS count FROM profesores p INNER JOIN tipos_contratos tp ON p.tipo_contrato = tp.id_tipo_contrato GROUP BY tp.tipo_contrato'

# Leemos la query y se la asignamos a un dataframe
df_profesor_tipo_contrato= pd.read_sql(query, conexion)

#Generamos la gráfica
alt.Chart(df_profesor_tipo_contrato).mark_bar().encode(
    alt.X("tipo_contrato:N", title="Tipo de contratos"),
    alt.Y("count:Q", title="Profesores asignados a un tipo de contrato"),
    alt.Color("tipo_contrato:N", title="Tipo de contratos"),
).properties(
    width=1500,
    height=500,
    title="¿Cuántos profesores tenemos por tipo de contrato?"
)


<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


### ¿Cuántas horas semanales imparte cada profesor?

In [33]:
# Consulta SQL (codigo de profesor, horas semanales)
query = 'SELECT codigo_profesor, CONCAT(nombres," ", apellido_paterno," ", apellido_materno) AS Profesor, horas_semanales FROM profesores ORDER BY horas_semanales DESC;'

# Leemos los datos de las horas semanales de cada profesor
df_horas_semanales = pd.read_sql(query, conexion)

#Imprimimos los resultados
print(df_horas_semanales)

  codigo_profesor                                     Profesor  \
0         1111112  maestro_002 appelido_pat002 appelido_mat002   
1         1111111  maestro_001 appelido_pat001 appelido_mat001   
2         1111113  maestro_003 appelido_pat003 appelido_mat003   
3         1111114  maestro_004 appelido_pat004 appelido_mat004   
4         1111120  maestro_010 appelido_pat010 appelido_mat010   
5         1111119  maestro_009 appelido_pat009 appelido_mat009   
6         1111118  maestro_008 appelido_pat008 appelido_mat008   
7         1111117  maestro_007 appelido_pat007 appelido_mat007   
8         1111116  maestro_006 appelido_pat006 appelido_mat006   
9         1111115  maestro_005 appelido_pat005 appelido_mat005   

   horas_semanales  
0              140  
1              120  
2              120  
3              120  
4               60  
5               50  
6               40  
7               30  
8               20  
9               12  


## ¿Cuántos profesores utilizaron recursos tecnológicos?

In [34]:
# Consulta SQL (resultado_encuesta_ultimo_ciclo, profesores)
query = """ SELECT COUNT(DISTINCT(codigo_profesor)) AS Profesores_Util_RT 
            FROM resultado_encuesta_ultimo_ciclo reuc 
            INNER JOIN respuesta_preguntas rp ON reuc.id_respuesta = rp.id_respuesta 
            WHERE no_pregunta = 10 and rp.id_respuesta = 1 OR rp.id_respuesta = 2 """

# Leer datos
df_carrera = pd.read_sql(query, conexion)

# Mostramos el contenido  del DataFrame
print(df_carrera)

   Profesores_Util_RT
0                   3


## ¿Cuántos profesores atendieron las necesidades de los estudiantes?

In [35]:
# Consulta SQL (resultado_encuesta_ultimo_ciclo, profesores)
query = """ SELECT COUNT(DISTINCT(codigo_profesor)) AS Profesores_att_ne 
            FROM resultado_encuesta_ultimo_ciclo reuc 
            INNER JOIN respuesta_preguntas rp ON reuc.id_respuesta = rp.id_respuesta 
            WHERE no_pregunta = 7 and rp.id_respuesta = 1 OR rp.id_respuesta = 2 """

# Leemos los datos de la base de datos
df_carrera = pd.read_sql(query, conexion)

# Mostramos el resultado
print(df_carrera)

   Profesores_att_ne
0                  3


---

# <a id="6">6. Comentarios adicionales<a> 🥽🥼

Durante el desarrollo del EDA, pude percatarme de varios atributos de tablas que no tenían el formato adecuado, como por ejemplo el atributo horas_semanales de la tabla profesores, el cual tenía como tipo varchar 100 y fue replantado por Int 4. Se abre la posibilidad de discutir sobre: Agregar la llave foránea, grado de estudio de la tabla profesores, que referencia a la tabla niveles_educativos, atributo: id_nivel. Además de  crear un campo de nacionalidad para estudiantes, referenciando a la tabla de nacionalidad, atributo id_nacionalidad.

Los grados académicos de los estudiantes en curso son: 7 estudiantes que tienen bachillerato, 5 licenciaturas, 3 maestrías y 7 técnicos superior universitario.
Artículo 33.

Del cuerpo estudiantil, 5 estudiantes están activos, mientras que 6 presentan el artículo 33, 4 el artículo 34, 1 está en licencia y 6 aprovecharon los convenios con universidades en el extranjero.

La universidad está compuesta por 11 estudiantes femeninas y 11 masculinos.

El cuerpo académico de esta universidad, que está compuesto por 10 profesores, todos están adscritos al Sistema Nacional de Investigadoras e Investigadores (SNII). De ellos, 4 están clasificados como SNII I, 5 como SNII II y 1 como SNII III. La distribución de género es equilibrada, con 5 profesores masculinos y 5 femeninos. Además, 4 de los profesores poseen el grado de doctorado, mientras que 6 tienen una maestría. En cuanto a la modalidad de contratación, 6 cuentan con un contrato definitivo, mientras que 4 tienen un contrato permanente o por tiempo indeterminado.

A continuación, se detalla la carga horaria de cada profesor:

| Código Profesor | Nombre del Profesor                             | Horas Semanales |
|-----------------|------------------------------------------------|------------------|
| 1111112         | maestro_002 apellido_pat002 apellido_mat002    | 140              |
| 1111111         | maestro_001 apellido_pat001 apellido_mat001    | 120              |
| 1111113         | maestro_003 apellido_pat003 apellido_mat003    | 120              |
| 1111114         | maestro_004 apellido_pat004 apellido_mat004    | 120              |
| 1111120         | maestro_010 apellido_pat010 apellido_mat010    | 60               |
| 1111119         | maestro_009 apellido_pat009 apellido_mat009    | 50               |
| 1111118         | maestro_008 apellido_pat008 apellido_mat008    | 40               |
| 1111117         | maestro_007 apellido_pat007 apellido_mat007    | 30               |
| 1111116         | maestro_006 apellido_pat006 apellido_mat006    | 20               |
| 1111115         | maestro_005 apellido_pat005 apellido_mat005    | 12               |


En la encuesta correspondiente al ciclo 2024A, solo 3 profesores incorporaron recursos tecnológicos en su enseñanza, mientras que otros 3 atendieron las necesidades de los estudiantes durante el curso.

---

# <a id="7">7. Referencias<a> 🎓

* J. VanderPlas. (2016). Python Data Science Handbook. O'Reilly Media.

* UBC MDS. (2019). Material público de los cursos Programación en Python para Data Science y Visualización de datos I del Master in Data Science de UBC.

* K. Katari. (Aug 21, 2020).Exploratory Data Analysis(EDA): Python. Towards Data Science.

* J.M. Reid. (Oct 14, 2021). 13 ways to access data in Python. Towards Data Science.

* A.K. Garg, V. Cuspinera-Contreras, Y. Qian. (Oct 2020). Bike Sharing Machine Learning Model, EDA section.

* Pure Storage, Inc. (2022). Datos estructurados frente a datos no estructurados.